In [3]:
import cv2
import numpy as np
#Dleap library to detect the face and then the eyes
import dlib
from math import hypot


#can have different indexes if there are multiple webcams
cap = cv2.VideoCapture(0)

#To detect the face
detector = dlib.get_frontal_face_detector()

#This dat file is basically used to detect points on the face. There are a total of 68 points in our face. The
#eyes ranging from 36. See landmarks_points.png for reference
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#we make this for top to bottom line because we have to draw the line from midpoint of two landmarks. (int)
#is used because pixels cant be divided or float.
def midpoint(p1,p2):
    return (int)((p1.x + p2.x)/2), (int)((p1.y+p2.y)/2)

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, facial_landmarks):
    
        #36 is the start point of eyes and 39 the end point. We draw a line from left to right and from top 
        #to bottom. When top to bottom of eyes length becomes 0, means, we are blinking eyes.
        
        left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
        right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
        center_top = midpoint(facial_landmarks.part(eye_points[1]),facial_landmarks.part(eye_points[1]))
        center_bottom = midpoint(facial_landmarks.part(eye_points[5]),facial_landmarks.part(eye_points[5]))
        
        
        #for horizontal line
        #hor_line = cv2.line(frame, left_point,right_point, (0,255,0), 2 )
        
        #for vertical line
        #ver_line = cv2.line(frame, center_top,center_bottom, (0,255,0), 2 )
        
        #hypot is used to calculate length of lines
        hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
        ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
        
        ratio = hor_line_length/ver_line_length
        
        return ratio

while True:
    _,frame = cap.read()
    
    #To obtain grayscale image. Grayscale images are lighter and that is why, we prefer them
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #Array where we have all the faces
    faces = detector(gray)
    
    for face in faces:
        #x,y = face.left(), face.top()
        #x1, y1 = face.right(), face.bottom()
        
        landmarks = predictor(gray,face)
        
        
        
        
        #Blinking detection
        left_eye_ratio = get_blinking_ratio([36,37,38,39,40,41],landmarks)
        right_eye_ratio = get_blinking_ratio([42,43,44,45,46,47],landmarks)
        
        blinking_ratio = (left_eye_ratio+ right_eye_ratio)/2
        
        if blinking_ratio>5.7:
            cv2.putText(frame,"Blinking",(50,150),font,7,(255,0,0)) 
    
    
        #Gaze detection
        #we need to specify the type of array which is done using np.int32. Numpy is basically used for working wit
        #arrays and mathematical operations.
        
        left_eye_region = np.array([[landmarks.part(36).x, landmarks.part(36).y],
                                   [landmarks.part(37).x, landmarks.part(37).y],
                                   [landmarks.part(38).x, landmarks.part(38).y],
                                   [landmarks.part(39).x, landmarks.part(39).y],
                                   [landmarks.part(40).x, landmarks.part(40).y],
                                   [landmarks.part(41).x, landmarks.part(41).y]] , np.int32)
        
        #We draw a polygon in the left eye region. True is whether polygon is closed or not.
        #Using polygon we can take exact region of the eye so that we can focus on only that portion
        
        #cv2.polylines(frame, [left_eye_region], True ,(0,0,255), 2)
        
        
        #We will create a mask here. A mask is simply a new blackwindow nothing else of the same size as the frame
        #The polygon that we created of the eye is placed on this mask
        height,width,_ = frame.shape
        mask = np.zeros((height,width),np.uint8)
        
        #only 255 because we want it to be white. Since, it will be placed on the mask which is black
        cv2.polylines(mask, [left_eye_region], True ,255, 2)
        cv2.fillPoly(mask,[left_eye_region],255)
        
        #This is done toobtain the eye part on the mask and not the threshold
        left_eye = cv2.bitwise_and(gray,gray,mask=mask)
        
        #We will try to take this polygon eye into different window
        #[:0] means that  If you have a 2-dimensional list/matrix/array, this notation will give you all the 
        #values in column 0 (from all rows).
        
        min_x = np.min(left_eye_region[:,0])
        max_x = np.max(left_eye_region[:,0])
        min_y = np.min(left_eye_region[:,1])
        max_y = np.max(left_eye_region[:,1])
        
        
        
        #We will create grayscale image of the eye and try creating a threshold and see if this threshold
        #is enough for us to use the eye for motion detection
        
        #gray_eye = cv2.cvtColor(eye,cv2.COLOR_BGR2GRAY)
        
        gray_eye = left_eye[min_y:max_y, min_x:max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 10, 255, cv2.THRESH_BINARY)
        
        
        #To resize the window we increase the size, none means we are not specifying any particular size
        #fx means current size multiplied by 4 here
        #Threshold is used to seperate the iris and pupil from thw white part of the eye
        threshold_eye = cv2.resize(threshold_eye,None, fx =5, fy=5)
        eye = cv2.resize(gray_eye,None, fx =4, fy=4)
        
        
        cv2.imshow("Left eye",left_eye)
        
        
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
        
    cv2.imshow("Frame",frame)
    
cap.release()
cv2.destroyAllWindows()